<a href="https://colab.research.google.com/github/ivkarla/connectivity_change/blob/main/notebooks/results_to_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Group results for all time frames to an Excel file with columns: subject, method, time frame, EN, hit, score1, network size


In [ ]:
import pandas as pd
import csv
from pickle import load

from google.colab import drive
drive.mount('/content/gdrive')
main_folder = "/content/gdrive/Shared drives/A&K/data/"

frames = ["NS_fixed27", "NN", "N60S", "NS", "NE"]


submeta = pd.read_excel(main_folder + 'subsmeta.xlsx')
subjects = list(submeta["sub"])
inres = load(open(main_folder + "inres_channels.pickle", "rb"))

nnodes,outcome={},{}
for si,s in enumerate(list(submeta["sub"])): nnodes[s],outcome[s]=list(submeta["n_channels_EDF"])[si],list(submeta["outcome"])[si]

S,M,EN,H,S1,L,F,O = [],[],[],[],[],[],[],[]
inres["MGM"] = ['F1', 'F2', 'F3', 'F4', 'F6', 'F7', 'F8', 'F12', 'F13', 'A8', 'O1', 'O2', 'O3', 'T5']

for frame in frames:
    if frame=="N60S": ext="ns"
    else: ext=frame.split('_')[0].lower()
    for sub in subjects:
        net_path = main_folder + 'reality_check/{}/nodes/'.format(frame)
        file = '{}-{}.csv'.format(sub,ext)
        r = pd.read_csv(net_path+file, header=None)
        with open(net_path+file, 'r') as csvfile:
                datareader = csv.reader(csvfile)
                for row in datareader:
                    if row!=[]:
                        met = row[0]

                        en = row[1].replace(" ","")[1:-1].split(",")
                        l = len(en)
                        els0 = list(el for elt in [ch[1:-1].split("-") for ch in en] for el in elt)
                        els = list(dict.fromkeys(els0))
                        hit = len(list(set(els) & set(inres[sub])))
                        score1 = hit/len(els) if len(els)>0 else 0

                        S.append(sub)
                        M.append(met)
                        EN.append(en)
                        H.append(hit)
                        S1.append(score1)
                        L.append(l)
                        F.append(frame)
                        O.append(outcome[sub])

                        # print(sub, met, hit, row[1], en, l)

info = pd.DataFrame({"subject":S, "outcome":O, "method":M, "time_frame":F, "EN":EN, "EN_size":L, "resection_hit":H, "score1":S1})
info.to_excel(main_folder + "results.xlsx")
